## Import libraries

In [17]:
import os
import numpy as np
import cv2
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [18]:
directory = "datasets/"

In [19]:
image_size = (244, 244)
classes = ["0", "1", "2"]

In [20]:
# Create an ImageDataGenerator instance with data augmentation settings
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    samplewise_center=True,
    samplewise_std_normalization=True,
)

In [21]:
training_data = []

def create_training_data():
    for category in classes:
        path = os.path.join(directory, category)
        class_num = classes.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, image_size)

                # Generate and store augmented images
                augmented_images = []
                augmented_images.append(new_array)  # Original image
                img_array_aug = new_array.reshape((1,) + new_array.shape)
                i = 0
                for batch in datagen.flow(img_array_aug, batch_size=1):
                    augmented_images.append(batch[0])
                    i += 1
                    if i >= 7:  # Generate 7 augmented images per input image
                        break

                for augmented_image in augmented_images:
                    image_hsv = cv2.cvtColor(augmented_image, cv2.COLOR_BGR2HSV)
                    training_data.append([image_hsv, class_num])

            except Exception as e:
                pass

In [22]:
create_training_data()

In [23]:
lenofimage = len(training_data)
print(lenofimage)

7800


In [24]:
X=[]
y=[]

for categories, label in training_data:
    X.append(categories)
    y.append(label)
X= np.array(X).reshape(lenofimage,-1)

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, )

# SVM Model

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
# Feature Scaling
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
# Hyperparameter Tuning using GridSearchCV
param_grid = {'C': [0.1, 1, 10],
              'kernel': ['linear', 'rbf', 'poly'],
              'gamma': ['scale', 'auto', 0.1, 1, 10]
              }

In [ ]:
# Train the SVM model
svm = SVC()
grid_search = GridSearchCV(svm, param_grid, cv=5)
grid_search.fit(X_train, y_train)

In [ ]:
# Get the best estimator and evaluate it
best_svm = grid_search.best_estimator_
y_test_pred = best_svm.predict(X_test)
accuracy = accuracy_score(y_test, y_test_pred)
print("Best SVM Parameters:", grid_search.best_params_)
print("Accuracy: {:.2f}%".format(accuracy * 100))

# Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
# Hyperparameter Tuning using GridSearchCV
param_grid = {'n_estimators': [50, 100, 200],
              'learning_rate': [0.01, 0.1, 0.2],
              'max_depth': [3, 4, 5]}

In [ ]:
gb = GradientBoostingClassifier()
grid_search = GridSearchCV(gb, param_grid, cv=5)
grid_search.fit(X_train, y_train)

In [ ]:
best_gb = grid_search.best_estimator_

# Test the model
y_test_pred = best_gb.predict(X_test)
accuracy = accuracy_score(y_test, y_test_pred)
print("Best Gradient Boosting Parameters:", grid_search.best_params_)
print("Accuracy: {:.2f}%".format(accuracy * 100))

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
# Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100, 200],  # You can adjust this range
    'max_depth': [None, 10, 20, 30]  # You can adjust this range
}

In [ ]:
# Create the Random Forest model
rf = RandomForestClassifier()

In [ ]:
# Create the GridSearchCV object
grid_search = GridSearchCV(rf, param_grid, cv=None)

In [ ]:
# Fit the GridSearchCV object to your data
grid_search.fit(X_train, y_train)

In [ ]:
# Get the best estimator and evaluate it
best_rf = grid_search.best_estimator_
y_test_pred = best_rf.predict(X_test)
accuracy = accuracy_score(y_test, y_test_pred)
print("Best Random Forest Parameters:", grid_search.best_params_)
print("Accuracy: {:.2f}%".format(accuracy * 100))

# KNN Model


In [27]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [28]:
# Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11]  # You can adjust this range
}

# Create the KNN model
knn = KNeighborsClassifier(n_neighbors = 5, weights='uniform', p = 2, algorithm='auto') #metric = 'minkowski',


In [29]:
# Create the GridSearchCV object
grid_search = GridSearchCV(knn, param_grid, cv=5)

In [ ]:
# Fit the GridSearchCV object to your data
grid_search.fit(X_train, y_train)

In [ ]:
# Get the best estimator and evaluate it
best_knn = grid_search.best_estimator_
y_test_pred = best_knn.predict(X_test)
accuracy = accuracy_score(y_test, y_test_pred)
print("Best KNN Parameters:", grid_search.best_params_)
print("Accuracy: {:.2f}%".format(accuracy * 100))

# Naive Bayes Model

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
# Define the parameter grid for hyperparameter tuning
param_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]  # You can adjust this range
}

In [ ]:
# Create the Gaussian Naive Bayes model
nb = GaussianNB()

In [ ]:
# Create the GridSearch object without cross-validation
grid_search = GridSearchCV(nb, param_grid, cv=None)  # Set cv=None for no cross-validation

In [ ]:
# Fit the GridSearch object to your data
grid_search.fit(X_train, y_train)

In [ ]:
# Get the best estimator and evaluate it
best_nb = grid_search.best_estimator_
y_test_pred = best_nb.predict(X_test)
accuracy = accuracy_score(y_test, y_test_pred)
print("Best Gaussian Naive Bayes Parameters:", grid_search.best_params_)
print("Accuracy: {:.2f}%".format(accuracy * 100))